In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import warnings
from datetime import *
warnings.simplefilter(action='ignore', category=FutureWarning)

In [136]:
#원하는 기간의 날짜를 리스트로 반환하는 함수
def date_range(start, end):
    '''
    startdate, enddate는 %Y%m%d 포맷으로 'YYYYMMDD' 형식
    '''
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [date.strftime("%Y.%m.%d") for date in pd.date_range(start, periods=(end-start).days+1)]
    return dates


def make_soup(url):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup




def last_botton_number(startdate,enddate,search_keyword,max_count_per_day):
    '''
    max_count_per_day = 크롤링할 일별 최대 뉴스의 개수
    
    
    네이버 뉴스페이지 구성

    시작페이지=1 step=10 으로 한 페이지당 10개의 뉴스를 노출시킨다.
    최대 노출 뉴스의 수는 400page 4000개의 뉴스를 노출시킨다. (그 이상은 불가능 필요하다면 검색 조건을 더 세부적으로 해야함)
    
    만약 검색한 키워드의 뉴스 총 노출수가 300개dlsep 100page를 url에 넣으면 알아서 300page로 들어가게된다

    따라서 마지막 페이지를 알아내려면 url에 page=4000을 넣으면 마지막 페이지로 이동할 수 있고
    그 soup에서 정규표현식을 통해 마지막 페이지의 숫자를 추출한다.



    날짜별로 반복:
        해당 날짜의 마지막 페이지 추출

        날짜, 마지막페이지번호, step 을 list에 추가

    return list
    '''

    url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_keyword}&sort=0&photo=0&field=0&pd=3&ds={date}&de={date}&cluster_rank=28&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{date_}to{date_},a:all&start={page}'

    #시작일 마감일 사이의 날짜들을 담은 리스트를 dates에 넣고 일 단위로 크롤링
    dates = date_range(startdate,enddate)
    
    date_lastpage_step = []

    for date in dates:  
        if max_count_per_day == -1:  
            url_ = url.format(search_keyword = search_keyword,
                            date = date,
                            date_ = date.replace('.',''),
                            page=4000)    # 네이버 뉴스페이지의 최대 노출 기사페이지 4000
            
            soup = make_soup(url_).select('a.btn') 
            
            try:    #검색 결과가 없는 경우 에러발생, 예외처리
                botton_count = int(re.findall('[0-9]+',str(soup[-1]).split(' ')[-1])[0]) * 10 - 9
                    
            except:
                print(date,'해당 일에 검색 결과가 존재하지 않습니다.')

        else:
            botton_count = max_count_per_day

        #페이지가 2이상 존재하지 않는 경우 step을 1로 설정
        step = 10 if botton_count > 10 else 1
        date_lastpage_step.append((date,botton_count,step))
    
    return date_lastpage_step



#['yna.co.kr', 'news1.kr', 'news.heraldcorp.com', 'gukjenews.com', 'view.asiae.co.kr', 'newspim.com', 'segye.com', 'news.kbs.co.kr','fnnews.com']
def main_text(url, news_agency):
    soup = make_soup(url)

    if news_agency == 'yna.co.kr':
        text = soup.select('#articleWrap > div.content01.scroll-article-zone01 > div > div > article')
        result = [i[4:-4]  for i in str(text).split('\n') if i[:3]=='<p>']

    elif news_agency == 'news.heraldcorp.com':
        soup = make_soup(url)
        text = soup.select('#articleText') 

    elif news_agency == 'gukjenews.com':
        text = soup.select('#article-view-content-div')
        result = [i[:-5].split('</p>')[0] if '</p>' in i[:-5] else i[:-5] for i in str(text).split('<p>')]

    elif news_agency == 'view.asiae.co.kr':
        text = soup.select('#txt_area')
        result = [i[6:].replace('<span>','').replace('</span>','') if i[:5] == '<br/>' else i.split('</div> ')[-1] for i in ' '.join([i for i in str(text).split('<p>')]).split('</p>')]

    elif news_agency == 'newspim.com':
        text = soup.select('#news-contents')
        result = [i[3:-4] for i in str(text).split('\n') if i[:3] == '<p>'] 

    elif news_agency == 'segye.com':
        text = soup.select('#article_txt > article')
        result = list(filter(None,[i[4:-4].replace('<br/>','') for i in str(text).split('\n') if i[:3] == '<p>']))[:-1]


    elif news_agency == 'news.kbs.co.kr':
        text = soup.select('#cont_newstext')
        result = [i for i in str(text).split('<br/>') if i != ''][1:]

    elif news_agency == 'fnnews.com':
        text = soup.select('#article_content')
        result = list(filter(None,[i[5:] for i in str(text).split('\n') if i[:5] == '<br/>']))


    return result



#start enddate default로 today()수정예정
def naver_news_url_crawling(search_keyword,max_count_per_day=-1,startdate='20220101',enddate='20220101'):
    '''

    max_count_per_day 하루 당 크롤링 할 최대 뉴스의 수 dafult시 (400,총 뉴스의 수) 중 작은값
    
    네이버 뉴스 웹은 page를 넘길 때 마다  (1, 11, 21, 31)순서로 바뀌고
    최대 page=4000 까지 지원한다. 페이지가 400이하일 때 page=4000을 넣으면 마지막 페이지가 나온다.
    매일 page=4000 url을 크롤링해서 마지막 페이지의 페이지버튼의 숫자를 통해 마지막 페이지를 크롤링하고
    for i in range(1, 총페이지수*10 -9, 10)으로 각 페이지마다 제목과 url 크롤링해서
    데이터프레임에 담아서 리턴 할 계획
    '''
    
    #검색하는 키워드의 띄어쓰기를 url상에 +으로 연결
    search_keyword = search_keyword.replace(' ','+')
    
    #네이버 뉴스 탭 url    
    url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_keyword}&sort=0&photo=0&field=0&pd=3&ds={date}&de={date}&cluster_rank=28&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{date_}to{date_},a:all&start={page}'

    #빈 데이터프레임을 만들고 크롤링한 데이터를 1일 단위로 concat해서 반환 할 계획
    result = []
    
    

                                #last_botton_number 파라미터는 클래스 선언하면서 추후에 수정
                                #max_count_per_day -1일 경우 최대치로 크롤링 해야하는데 수정해야함
    for date, botton_count, step in last_botton_number(startdate,enddate,search_keyword,max_count_per_day):
        for page in range(1,botton_count+1,step): 
            url_ = url.format(search_keyword = search_keyword,
                              date = date,
                              date_= date.replace('.',''),
                              page = page)

            soup = make_soup(url_).select('a.news_tit')

            #한 페이지에 최대 노출 기사수 10개 1일 단위로 크롤링을 하니
            a_page_ulr = [html['href'] for html in soup[:min(10,max_count_per_day)]]
            a_page_title = [html['title'] for html in soup[:min(10,max_count_per_day)]]
            a_page_date = [date] * len(a_page_title)

            result.append(pd.DataFrame({'date':a_page_date, 'title':a_page_title, 'url':a_page_ulr}))
    
    result = pd.concat(result,ignore_index=True).drop_duplicates()

    # url에서 신문사의 정보를 유추할 수 있는 부분을 추출
    result['news_agency'] = result['url'].apply(lambda x : x.split('/')[2]).apply(lambda x : x.replace('www.',''))
    

    return result




In [3]:
df = naver_news_url_crawling(search_keyword='교통',startdate='20220401',enddate='20220402')
df

,date,title,url,news_agency
0,2022.04.01,"서울시, 양화교 접속램프 교통통제…내년 2월까지",http://www.newsis.com/view/?id=NISX20220331_00...,newsis.com
1,2022.04.01,"교통안전공단, 변환빔 전조등시험기 특별점검",http://news.mt.co.kr/mtview.php?no=20220401112...,news.mt.co.kr
2,2022.04.01,장애인 이동권 논란에… 서울교통공사 “2024년까지 ‘1역사 1동선’ 확보”,http://www.segye.com/content/html/2022/04/01/2...,segye.com
3,2022.04.01,"정부 대표단, 싱가포르·말레이 찾아 교통인프라 협력 외교전",http://yna.kr/AKR20220331163900003?did=1195m,yna.kr
4,2022.04.01,오늘부터 접종 해외입국자 대중교통 이용 가능,https://www.sedaily.com/NewsView/264IVVWAPR,sedaily.com
...,...,...,...,...
2280,2022.04.02,"[6.1지선] 이상열 남양주시장 예비후보, 개소식과 함께 힘찬 출발",https://www.onews.tv/news/articleView.html?idx...,onews.tv
2281,2022.04.02,"남양주남부경찰서, 이륜차 불법개조 합동단속..12대 적발",https://www.onews.tv/news/articleView.html?idx...,onews.tv
2282,2022.04.02,"""거리두기 장기화, 통행량 감소 효과 낮아…통행 집중도 분산 필요""",https://www.etoday.co.kr/news/view/2119884,etoday.co.kr
2283,2022.04.02,후지산 분화 3시간 내 용암류 도달 지역에 11만 명…기존 예상치 7배,http://www.newshankuk.com/news/content.asp?new...,newshankuk.com


In [146]:
df2 = df[:100]
#[i for i in ]
[i for i in df2[['url','news_agency']]]


['url', 'news_agency']

In [4]:
df['news_agency'] = df['news_agency'].apply(lambda x : 'yna.co.kr' if x == 'yna.kr' else x)

In [99]:
top10 = df['news_agency'].value_counts(normalize=True)[:30] * 100
top10.cumsum()

newsis.com              3.110704
yna.co.kr               5.489478
news1.kr                7.822507
news.heraldcorp.com     9.789570
gukjenews.com          11.390668
view.asiae.co.kr       12.991766
newspim.com            14.592864
segye.com              16.056725
news.kbs.co.kr         17.520586
fnnews.com             18.938701
news.mt.co.kr          20.128088
news.mk.co.kr          21.317475
breaknews.com          22.369625
edaily.co.kr           23.376029
hankyung.com           24.336688
ajunews.com            25.297347
nocutnews.co.kr        26.212260
chosun.com             27.127173
etoday.co.kr           27.996340
kukinews.com           28.819762
ytn.co.kr              29.643184
sedaily.com            30.466606
shinailbo.co.kr        31.244282
asiatoday.co.kr        32.021958
dailian.co.kr          32.799634
seoul.co.kr            33.577310
moneys.mt.co.kr        34.354986
dnews.co.kr            35.086917
wikitree.co.kr         35.818847
sisamagazine.co.kr     36.550778
Name: news

In [6]:
top3 = top10.index[1:11]
top3

Index(['yna.co.kr', 'news1.kr', 'news.heraldcorp.com', 'gukjenews.com',
       'view.asiae.co.kr', 'newspim.com', 'segye.com', 'news.kbs.co.kr',
       'fnnews.com', 'news.mt.co.kr'],
      dtype='object')

In [7]:
df1 = df[df['news_agency'].isin(top3)].reset_index(drop=True)
df1

,date,title,url,news_agency
0,2022.04.01,"교통안전공단, 변환빔 전조등시험기 특별점검",http://news.mt.co.kr/mtview.php?no=20220401112...,news.mt.co.kr
1,2022.04.01,장애인 이동권 논란에… 서울교통공사 “2024년까지 ‘1역사 1동선’ 확보”,http://www.segye.com/content/html/2022/04/01/2...,segye.com
2,2022.04.01,"정부 대표단, 싱가포르·말레이 찾아 교통인프라 협력 외교전",http://yna.kr/AKR20220331163900003?did=1195m,yna.co.kr
3,2022.04.01,"""치솟는 기름값이 무서워요""…카풀·대중교통에 '원정 주유'까지",http://yna.kr/AKR20220401100700061?did=1195m,yna.co.kr
4,2022.04.01,"경상남도, 교통약자 이동서비스 확대",https://news.kbs.co.kr/news/view.do?ncd=542970...,news.kbs.co.kr
...,...,...,...,...
367,2022.04.02,"2월 주택 매매거래량 총 43,179건 집계→전월 대비 3.5% 증가",https://www.gukjenews.com/news/articleView.htm...,gukjenews.com
368,2022.04.02,"[강원 날씨] 강원도 산지 영하권 날씨→한쪽은 봄꽃, 한쪽은 얼음",https://www.gukjenews.com/news/articleView.htm...,gukjenews.com
369,2022.04.02,"서울시, 마포로1구역 제34지구 재개발 정비계획 변경(안)' 수정가결",https://www.gukjenews.com/news/articleView.htm...,gukjenews.com
370,2022.04.02,"구자천 창원상의 회장 “공공기관 2차 지방이전, 국정과제로 채택돼야”""",https://view.asiae.co.kr/article/2022040212000...,view.asiae.co.kr


In [ ]:
#['yna.co.kr', 'news1.kr', 'news.heraldcorp.com', 'gukjenews.com', 'view.asiae.co.kr', 'newspim.com', 'segye.com', 'news.kbs.co.kr','fnnews.com', 'news.mt.co.kr']
def main_text(url, news_agency):
    soup = make_soup(url)

    if news_agency == 'yna.co.kr':
        text = soup.select('#articleWrap > div.content01.scroll-article-zone01 > div > div > article')
        result = [i[4:-4]  for i in str(text).split('\n') if i[:3]=='<p>']

    elif news_agency == 'news.heraldcorp.com':
        soup = make_soup(url)
        text = soup.select('#articleText') 

    elif news_agency == 'gukjenews.com':
        text = soup.select('#article-view-content-div')
        result = [i[:-5].split('</p>')[0] if '</p>' in i[:-5] else i[:-5] for i in str(text).split('<p>')]

    elif news_agency == 'view.asiae.co.kr':
        text = soup.select('#txt_area')
        result = [i[6:].replace('<span>','').replace('</span>','') if i[:5] == '<br/>' else i.split('</div> ')[-1] for i in ' '.join([i for i in str(text).split('<p>')]).split('</p>')]

    elif news_agency == 'newspim.com':
        text = soup.select('#news-contents')
        result = [i[3:-4] for i in str(text).split('\n') if i[:3] == '<p>'] 

    elif news_agency == 'segye.com':
        text = soup.select('#article_txt > article')
        result = list(filter(None,[i[4:-4].replace('<br/>','') for i in str(text).split('\n') if i[:3] == '<p>']))[:-1]


    elif news_agency == 'news.kbs.co.kr':
        text = soup.select('#cont_newstext')
        result = [i for i in str(text).split('<br/>') if i != ''][1:]

    elif news_agency == 'fnnews.com':
        text = soup.select('#article_content')
        result = list(filter(None,[i[5:] for i in str(text).split('\n') if i[:5] == '<br/>']))


    return result

In [80]:
#segye.com
url = 'http://www.segye.com/content/html/2022/03/31/20220331520428.html?OutUrl=naver'
soup = make_soup(url)
text = soup.select('#article_txt > article')
result = list(filter(None,[i[4:-4].replace('<br/>','') for i in str(text).split('\n') if i[:3] == '<p>']))[:-1]
return result

['전지역 여성인재가 시정의 주요 정책과정에 참여할 수 있는 기회가 확대된다. 대전시는 정책 의사결정에서 여성 대표성을 높이기 위해 다양한 분야에서 여성인재를 집중 발굴하는 ‘대전 여성인재 데이터베이스(DB)’를 구축한다고 31일 밝혔다.',
 '에 따르면 2020년 기준 대전 성평등지수(100점 만점)는 복지(98.6점), 직업훈련(95.2점), 경제활동(76.6점), 문화정보(85.4점) 등 대부분 분야에서 여성 비중이 높은 수준을 유지하고 있었으나 의사결정 분야에선 48.5점으로 상대적으로 낮았다.',
 '사결정 분야의 4개 지표를 보면 ‘지방자치단체위원회 위촉위원 성비’(72.5점)를 제외한 ‘광역·기초 의원 비율 성비’(50.9점), ‘5급 이상 공무원 비율 성비’(31.2점), ‘관리자 비율 성비’(39.4점) 등에서 점수가 현저히 낮았다.',
 '는 DB에 등록된 여성인재를 공공기관 임원, 정책자문단, 각종 위원회 위원 등으로 추천하고 전문분야별 연구, 자문, 강사 초빙 등에 활용될 수 있도록 지원할 방침이다.',
 '는 지난해 여성인재 DB에 기업인, 교육인, 법조인, 의료인, 문화예술 등 분야별 전문 여성인재 682명을 등록했다.',
 '성인재 DB 구축 이후 시 운영위원회의 여성 참여 비율은 늘고 있는 것으로 나타났다.',
 '성 비율이 40% 이상인 위원회는 2019년 118개(67.8%)에서 지난해 122개(72.2%)로 4.4%포인트 상승했다. 올해는 상대적으로 인재풀이 저조한 과학기술·건설·교통 분야 등을 중점적으로 확충할 계획이다. 관심 있는 여성은 대전 여성인재 DB 홈페이지에서 자신의 전문영역, 활동사항 등을 직접 등록할 수 있다.',
 '는 DB에 등록된 여성인재를 중심으로 역량 개발을 도모하는 맞춤형 여성인재아카데미 사업과 교육 프로그램 등 여성의 경제적 자립과 성평등 사회 실현을 위한 정책도 추진할 예정이다.',
 '경희 대전시 성인지정책담당관은 “여성의 사회 참여 확대는 여성의 지위 향상뿐 아니라 국가경쟁력 제고를 위해 적

In [59]:
#view.asiae.co.kr
url = 'https://view.asiae.co.kr/article/2022040118065881860'
soup = make_soup(url)
text = soup.select('#txt_area')
result = [i[6:].replace('<span>','').replace('</span>','') if i[:5] == '<br/>' else i.split('</div> ')[-1] for i in ' '.join([i for i in str(text).split('<p>')]).split('</p>')]
result

['<!-- 기사 부제목 영역// -->\n<!-- 본문 -->\n<div class="article_photo">\n<a class="img_link" href="//www.asiae.co.kr/news/img_view.htm?img=2022040118060513302_1648803964.jpg" title="">\n<img alt="" src="https://cphoto.asiae.co.kr/listimglink/1/2022040118060513302_1648803964.jpg"/>\n</a>\n<p class="txt">경남경찰청 고속도로순찰대 제6지구대.',
 '\n<div class="article_ad"><ul><li><script src="https://a1.asiae.co.kr/RealMedia/ads/adstream_jx.ads/www.asiae.co.kr/society@x30" type="text/javascript"></script></li></ul><div class="bl_ad">AD</div></div>\n<a class="img_figure" href="//www.asiae.co.kr/news/img_view.htm?img=2022040118060513302_1648803964.jpg" title="">\n<span class="blind">썝蹂몃낫湲 븘씠肄</span>\n</a>\n</div>\n<br/> [아시아경제 영남취재본부 이세령 기자] 경남경찰청 고속도로순찰대 제6 지구대가 1일부터 한 달간 고속도로 화물차, 과속·난폭운전 등 교통법규 위반행위를 집중 단속한다.',
 '지구대는 고속순찰차, 암행순찰차 등 가용경력을 투입해 지정차로 위반, 안전거리 미확보 등 화물차량 교통법규 위반행위와 모든 차종의 과속·난폭운전을 막고 봄 행락철 고속도로 대형교통사고를 예방한다.',
 '과속으로 일어날 수 있는 교통사고를 예방하고자 고정형 이동식 단속카메라와 함께 암행순찰차 내 차량탑재형 과속단속카메라를 이용해, 주행 중 단속 등을 강화할 

In [31]:
#	newspim.com
url = 'http://www.newspim.com/news/view/20220401000711'
soup = make_soup(url)
text = soup.select('#news-contents')
result = [i[3:-4] for i in str(text).split('\n') if i[:3] == '<p>']
result


['원일희 대통령직인수위원회 수석부대변인은 1일 오후 2시 서울 통의동 인수위에서 열린 브리핑에서 "수요가 있는 곳에 신속한 주택 공급을 이행하기 위해 국토부와 서울시가 도심주택공급실행TF을 구성했다"며 "당선인이 공약으로 발표한 250만호 공급이라는 큰 목표를 달성하기 위해 구체적이고 상세한 계획이 구성될 예정"이라고 밝혔다.',
 '구체적으로TF는 ▲역세권 첫 집 주택·청년 원가 주택 ▲수요가 높은 서울시 내 주택 공급 로드맵 마련 ▲선도사업 대상지 발굴 및 사업계획 수립 등을 마련할 계획이다.',
 '원 부대변인은 "국토부는 필요한 제도를 개선하고 서울시는 인허가 등 신속한 절차를 이행하기 때문에 두 기관이 TF를 마련했다"고 설명했다.',
 '이어 "지난달 30일 부동산TF에서 나온 의견을 실행하는 1호 조치라고 이해하시면 되겠다"고 했다.',
 'adelante@newspim.com</p></d']

In [32]:
#news.heraldcorp.com
url = 'http://news.heraldcorp.com/view.php?ud=20220401000017'
soup = make_soup(url)
text = soup.select('#articleText')
result = [i.replace('</p>','') for i in str(text).split('<p>')[1:]]
result

['[헤럴드경제=신동윤 기자] 서방권에서 가장 먼저 신종 코로나바이러스 감염증(코로나19) 타격을 받은 이탈리아가 2년여 만에 본격적인 일상 회복의 첫발을 내디딘다.',
 '이탈리아는 31일(현지시간)부로 보건비상사태를 해제했다. 전 세계적으로 코로나19가 퍼지기 시작한 2020년 1월 처음 도입된 이후 약 2년 2개월 만이다.',
 '보건비상사태 해제는 코로나19 비상 체제를 마무리하고 평시로 돌아가는 신호탄으로 해석된다.',
 '그동안 코로나19 대책을 자문해온 정부 과학기술위원회와 대응 실행 기구인 코로나19 비상대책위원회도 이날부로 해산했다. 향후 모든 코로나19 관련 대책 수립·대응은 보건부로 일원화된다.',
 '보건비상사태 해제에 맞춰 4월 1일부터 방역 규제 조처도 단계적으로 풀린다.',
 '음식점·바의 야외 테이블과 박물관·미술관·우체국·은행 등의 공공시설 등에 대해선 그린 패스(방역패스)가 더는 필요치 않다.',
 '시내·시외·장거리 대중 교통수단 역시 그린 패스 대상에서 제외된다.',
 '또한 코로나19 확진자와 접촉한 사람도 백신 접종 여부와 관계없이 격리 의무에서 벗어난다. 열흘간 자가 관찰을 하고, 외출 시 우리나라의 KF94에 해당하는 FFP2 마스크 착용하는 것으로 충분하다.',
 '50세 이상 백신 미접종자 역시 코로나19 음성 확인증만 제출하면 일터에 출근할 수 있다.',
 '그동안에는 ‘슈퍼 그린패스’ 적용에 따라 백신을 맞지 않으면 출근 자체가 불가능했을뿐더러 무단결근 처리돼 그에 해당하는 급여도 받지 못했다.',
 '일선 학교의 경우 코로나19 확진자 외에 전원 대면 수업을 원칙으로 하며, 축구경기장·콘서트에는 관중·관객 100% 수용이 가능해진다.',
 '다만, 실내 음식점·바에 대한 그린패스 적용과 실내 공공장소에서의 FFP2 마스크 착용 의무는 4월 말까지 유지된다.',
 '2020년 2월 코로나19 지역 감염이 처음 확인된 이탈리아는 31일 현재 누적 확진자 1464만2354명, 총 사망자 15만9383명을 각각 기록

In [33]:
#kbs
url = 'https://news.kbs.co.kr/news/view.do?ncd=5435528&ref=A'
soup = make_soup(url)
text = soup.select('#cont_newstext')
result = [i for i in str(text).split('<br/>') if i != ''][1:]
result

[' 지난 2년 간 코로나 19로 인해 각국을 오가는 항공기 이동량과 공장 가동률 등이 급격히 감소하면서 대기질은 상대적으로 개선됐을 것이라는 전망이 많았습니다.',
 ' 하지만 최근 세계보건기구 WHO가  각국의 대기질을 분석한 결과,  코로나 대유행 전보다 대기질이 더 나빠진 것으로 드러났습니다.',
 ' 결국 화석연료 의존도가 여전히 높다는 지적입니다.',
 ' 오늘 이 소식, 지구촌 돋보기에서 임민지 기자와 함께 자세히 들여다보겠습니다.',
 ' 임 기자,  세계보건기구의 분석 결과에 따르면 대기질 지수가 얼마나 심각한가요?',
 ' [기자]',
 ' WHO 세계보건기구가 지난 4일, 세계 117개국 6천 개 도시의  공기 질을 분석했는데요.',
 ' 분석 결과, 전 세계 인구의 99%가  건강에 해로운 공기를 마시고 있는 것으로 나타났습니다. ',
 ' 지난 2018년 같은 조사에서는  건강에 해로운 공기를 마시는 세계 인구가 90%였었는데, 4년 사이에 상황이 더 악화된 겁니다.',
 ' 여전히 화석연로에 대한  의존도가 높기 때문입니다.',
 ' WHO에서는 심지어 우리는 그동안 깨끗하고 건강한 공기에 투자하기보다 환경을 오염시키는 데 더 많은 투자를 했다고 자조적으로 평가하고 있습니다.',
 '[마리아 네이라/WHO 환경·기후변화·보건국 국장 : "WHO는 수년 동안 대기 오염 노출이 국민의 건강을 해친다는 것을 경고해왔습니다. 매년 7백만 명이 넘는 사람들이 대기 오염에 대한 노출로 숨집니다."]',
 ' [앵커]',
 ' 결국, 탄소 배출을 최소화하는 청정 에너지원을 개발하는 것이 급선무라는 생각이 드는데  그런 움직임은 있습니까?',
 ' [기자]',
 ' 네, 기후 변화의 주요 원인이 탄소 배출량을 줄이지 못하는 거죠.',
 ' 다행인건 상황의 심각성을 깨달으면서 다양한 노력이 전 지구적으로 진행되고 있다는 점입니다. ',
 ' 우선 영국 과학자들의  최근 발표 내용이 눈길을 끕니다.',
 ' 핵분열을 이용한 원자력 에너지보다 훨씬 안전하

In [34]:
#fnnews
url = 'http://www.fnnews.com/news/202204011447482503'
soup = make_soup(url)
text = soup.select('#article_content')
result = list(filter(None,[i[5:] for i in str(text).split('\n') if i[:5] == '<br/>']))
result

['[파이낸셜뉴스] 대통령직인수위원회(인수위)는 1일, 국토교통부와 서울시가 도심 주택공급 실행을 위한 태스크포스(TF)를 구성했다고 밝혔다.',
 '원일희 인수위 수석부대변인은 이날 인수위 기자회견장에서 열린 브리핑에서 "수요가 있는 곳에 신속한 주택공급을 이행하는 것이 목표"라고 밝혔다.',
 '이는 3월 30일 인수위 부동산TF에서 나온 의견을 실행하는 1호 조치라고 설명하며 "당선인이 공약으로 발표한 250만호 공급 목표 공급을 달성하기 위한 구체적 사업계획을 마련할 예정"이라고 말했다.',
 '원 수석부대변인은 "TF는 첫 번째로 역세권 첫 집 주택과 청년 원가주택, 두 번째는 수요 높은 서울 시내 주택공급 로드맵 마련, 세 번째로는 선도사업 대상지 발굴 및 사업계획을 수립 중"이라며 "국토부는 필요한 제도를 개선하고 서울시는 인허가 등 신속절차를 이행하기 위해 두 기관이 협력했다"고 밝혔다.']

In [35]:
#yna
url = 'https://www.yna.co.kr/view/AKR20220501015600003?input=1195m'
soup = make_soup(url)
text = soup.select('#articleWrap > div.content01.scroll-article-zone01 > div > div > article')
result = [i[4:-4]  for i in str(text).split('\n') if i[:3]=='<p>']
result

['(서울=연합뉴스) 황희경 기자 = 지난달 신종 코로나바이러스 감염증(코로나19) 방지를 위한 사회적 거리두기가 해제되면서 출근용 정장이나 나들이용 카메라 등을 찾는 사람들이 늘어나고 있다. ',
 '1일 코오롱인더스트리FnC부문에 따르면 지난달 나들이와 외출 때 가볍게 들 수 있는 크로스백이나 남성 캐주얼 재킷, 정장류 매출이 증가했다.',
 "여성 핸드백 브랜드 '쿠론'의 크로스백인 버베나 백은 4월을 기점으로 판매율이 64% 이상을 기록했다. ",
 '코오롱FnC 관계자는 "재택근무에서 출근으로 전환되면서 외출복 구입이 늘어났고, 포근해진 날씨 영향으로 나들이 및 야외 활동에 필요한 아이템이 인기를 얻고 있다"면서 "가정의 달인 5월로 접어들면서 더 많은 소비가 이뤄질 것"이라고 말했다.<br/>',
 '인기 가전 트렌드도 바뀌고 있다.',
 '지난 2년간 집에서 머무는 시간이 늘어나면서 커피머신이나 오디오 같은 실내용 가전 상품들이 주목을 받았다면 최근에는 나들이와 해외여행 등 야외활동을 준비하는 사람들이 증가하면서 카메라와 뷰티 가전 수요가 늘었다.',
 '실제 롯데백화점에서는 지난달 카메라와 뷰티 가전 매출이 전년보다 각각 2배 이상 늘어났다. ',
 '롯데백화점은 이런 변화에 맞춰 잠실점에서 미러리스 카메라와 렌즈를 3일간 무료로 체험해 불 수 있는 행사를 하고 나들이족 공략에 나섰다.',
 '또 실외 마스크 착용 의무 해제에 맞춰 피부관리기기 상품들을 가정의달 선물로 집중적으로 홍보하고 있다. ',
 '이동현 롯데백화점 H&amp;E 부문장은 "이번 가정의 달에는 전통적인 가전 상품과 함께 엔데믹(endemic·풍토병으로 굳어진 감염병) 트렌드에 맞는 소형 가전 상품들도 주목받을 것"이라고 예상했다. ',
 '이마트에서는 사회적 거리두기가 해제된 지난달 18일 이후 28일까지 건강기능식품 매출이 지난해 같은 기간보다 13.3% 늘었다. ',
 '홍삼 건강기능식품 매출은 43.8%, 유산균 등 장 건강기능식품 매출은 29.9% 각각 증가했다. '